In [16]:
import pandas as pd
import numpy as np
import warnings

from operator import itemgetter
from sklearn.experimental import enable_iterative_imputer 
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import (GradientBoostingRegressor, GradientBoostingClassifier)

warnings.filterwarnings('ignore')

In [17]:
data = pd.read_csv("../NYC_property_sales(2).csv")

# Data Cleaning

In [20]:
def find_missing_percent(data):
    miss_df = pd.DataFrame({'ColumnName':[],'TotalMissingVals':[],'PercentMissing':[]})
    for col in data.columns:
        sum_miss_val = data[col].isnull().sum()
        percent_miss_val = round((sum_miss_val/data.shape[0])*100,2)
        miss_df = miss_df.append(dict(zip(miss_df.columns,[col,sum_miss_val,percent_miss_val])),ignore_index=True)
    return miss_df

In [26]:
missing_dataframe = find_missing_percent(data)

display(missing_dataframe[missing_dataframe['PercentMissing']>0.0])
print("\n")
print(f"Number of columns with missing values:{str(missing_dataframe[missing_dataframe['PercentMissing']>0.0].shape[0])}")

,ColumnName,TotalMissingVals,PercentMissing
8,apartment_number,26109.0,68.39
9,zip_code,33.0,0.09




Number of columns with missing values:2
